In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.gridspec as gridspec

In [32]:
x, y, b1, b2, b3 = np.loadtxt('/Users/yao/Documents/Data/AMRVAC/test1/visit_b1_b2_b3_0010.xyz',
                        unpack=True,skiprows=2,usecols=[1,2,4,5,6])

print(x.size)


332508


In [33]:
Nx=np.unique(x).size
Ny=np.unique(y).size

print(x.size)
print(Nx, Ny)

332508
1001 321


In [29]:
Nx=np.unique(x).size
Ny=np.unique(y).size

print(x.size)
print(Nx, Ny)

341220
1021 321


In [ ]:
5: 
332508
1001 321

In [17]:
mask = np.isnan(y)
print(mask)
print(np.any(mask))
print(np.sum(mask))
print(np.where(mask))

[False False False ... False False False]
False
0
(array([], dtype=int64),)


In [20]:
np.unique(x).size

1001

In [21]:
xmin=np.min(x)
xmax=np.max(x)
ymin=np.min(y)
ymax=np.max(y)

In [22]:
xyz_combined=np.vstack((x, y, b1, b2, b3)).T


In [23]:
xyz_sorted=xyz_combined[np.lexsort((xyz_combined[:, 1], xyz_combined[:, 0]))]


In [24]:
unique_xyz=np.unique(xyz_sorted, axis=0)


In [25]:
x, y, b1, b2, b3 =unique_xyz.T


In [26]:
x=x.reshape(Nx,Ny)

ValueError: cannot reshape array of size 276404 into shape (1001,321)